# QSwitch usage

In [2]:
from time import sleep as sleep_s
import pprint
pp = pprint.PrettyPrinter()
import numpy as np
import qcodes
from qcodes_contrib_drivers.drivers.QDevil import QSwitch
qswitch_addr = '192.168.8.20'
qswitch = QSwitch.QSwitch('switch', visalib='@py', address=f'TCPIP::{qswitch_addr}::5025::SOCKET')

Connected to: Quantum Machines QSwitch (serial:1, firmware:0.160) in 0.11s


In [3]:
qswitch.reset()
qswitch.state()

'(@1!0:24!0)'

State as a Python array:

In [4]:
qswitch.all_closed_relays()

[(1, 0),
 (2, 0),
 (3, 0),
 (4, 0),
 (5, 0),
 (6, 0),
 (7, 0),
 (8, 0),
 (9, 0),
 (10, 0),
 (11, 0),
 (12, 0),
 (13, 0),
 (14, 0),
 (15, 0),
 (16, 0),
 (17, 0),
 (18, 0),
 (19, 0),
 (20, 0),
 (21, 0),
 (22, 0),
 (23, 0),
 (24, 0)]

Beep on error:

In [5]:
qswitch.error_indicator('on')

## Manipulation by numbers

Connect and unground line 23:

In [6]:
qswitch.close_relay(23, 9)
qswitch.open_relay(23, 0)
qswitch.state()

'(@1!0:22!0,24!0,23!9)'

Tap off line 23 through BNC 2:

In [7]:
qswitch.close_relay(23, 2)
qswitch.state()

'(@1!0:22!0,24!0,23!9,23!2)'

## Arrangements

Lines and break-out BNCs can be given names.  There are specialised functions for manipulating relays by name.  For convenience, the default name of a line or BNC is just its number, so

In [8]:
qswitch.connect('23')
qswitch.unground('23')
qswitch.breakout('23', '2')

is equivalent to

In [9]:
qswitch.close_relay(23, 9)
qswitch.open_relay(23, 0)
qswitch.close_relay(23, 2)

To give names to lines or BNCs, use `arrange`:

In [10]:
qswitch.arrange(
    # BNC 2 connected to DMM, BNC 7 to VNA
    breakouts={'DMM': 5, 'VNA': 7},
    # Give names to lines 23 & 5
    lines={'plunger': 23, 'sensor': 5})

In [11]:
qswitch.connect('sensor')
qswitch.breakout('sensor', 'DMM')
qswitch.unground('sensor')
qswitch.errors()

'0,"No error"'

In [12]:
qswitch.state()

'(@1!0:4!0,6!0:22!0,24!0,5!9,23!9,23!2,5!5)'

Manipulate several lines at once:

In [12]:
qswitch.unground(map(str,[1,2,3,4,5,8,10,11,12]))
qswitch.disconnect(map(str,range(14, 21+1)))
qswitch.state()

'(@6!0:7!0,9!0,13!0:22!0,24!0,23!2,5!5,5!9,23!9)'

## Monitors

There is a pseudo parameter dedicated to monitoring:

In [13]:
qswitch.overview()

{'6': ['grounded'],
 '7': ['grounded'],
 '9': ['grounded'],
 '13': ['grounded'],
 '14': ['grounded'],
 '15': ['grounded'],
 '16': ['grounded'],
 '17': ['grounded'],
 '18': ['grounded'],
 '19': ['grounded'],
 '20': ['grounded'],
 '21': ['grounded'],
 '22': ['grounded'],
 '24': ['grounded'],
 'plunger': ['breakout 2', 'connected'],
 'sensor': ['breakout DMM', 'connected']}

In [14]:
monitor = qcodes.Monitor(qswitch.overview)

## Autosave

When turning on autosave, the state will be restored over restart.

In [16]:
qswitch.auto_save('on')
qswitch.auto_save('off')